## Iniciando o Pyspark

In [20]:
!pip install pyspark #==3.3.1

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2024-01-24 12:38:05--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 18.205.222.128, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.3’

ngrok-stable-linux- 100%[===================>]  13.28M  17.9MB/s    in 0.7s    

2024-01-24 12:38:06 (17.9 MB/s) - ‘ngrok-stable-linux-amd64.zip.3’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Iniciando Sessão Spark

In [21]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

# ConfigureSparkUI
conf = SparkConf().set('spark.ui.port', '4050')
sc = SparkContext(conf=conf)
sc.stop()

spark = (
    SparkSession.builder                  # Método da classe que constrói a sessão spark
      .appName("Exercicios Spark")  # Nome do App Spark
      .getOrCreate())                     # Verifica se há uma sessão ativa, e se não há, cria uma nova sessão

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Exercicios Spark, master=local[*]) created by getOrCreate at <ipython-input-4-65b1d40b5429>:12 

In [22]:
get_ipython().system_raw('./ngrok http 4050 &')

In [23]:
!curl -s http://localhost:4040/api/tunnels

In [24]:
get_ipython().system_raw('./ngrok config add-authtoken 2KBeQEmmd1YNlQ86GGKf3KFOkb3_6sQH7JEnvEhDxwn9A7WnT')

In [25]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, TimestampType


schema_base_pix = StructType([
    StructField('id', IntegerType()),
    StructField('valor', DoubleType()),
    StructField('parte_debitada_nome', StringType()),
    StructField('parte_debitada_conta', StringType()),
    StructField('parte_debitada_banco', StringType()),
    StructField('parte_creditada_nome', StringType()),
    StructField('parte_creditada_conta', StringType()),
    StructField('parte_creditada_banco', StringType()),
    StructField('chave_pix_tipo', StringType()),
    StructField('chave_pix_valor', StringType()),
    StructField('data_transacao', TimestampType())
])

caminho_csv = "./base_de_dados.csv"

df = spark.read.csv(
      path=caminho_csv,
      header=True,
      sep=";",
      schema=schema_base_pix,
      timestampFormat="dd/MM/yyyy HH:mm"
)

In [26]:
df.show()

+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+---------------+-------------------+
| id|   valor| parte_debitada_nome|parte_debitada_conta|parte_debitada_banco|parte_creditada_nome|parte_creditada_conta|parte_creditada_banco|chave_pix_tipo|chave_pix_valor|     data_transacao|
+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+---------------+-------------------+
|  1|    9.93|Dra. Ana Carolina...|            79470453|              Nubank|       Maysa da Cruz|             67162333|                 Itau|           cpf|     8439752610|2022-02-18 13:28:00|
|  2|   15.38|        Ana Caldeira|            19689668|                Itau|        Evelyn Sales|             60005091|             Bradesco|           cpf|    27145380617|2022-04-08 01:47:00|
|  3|   57.58|    Arthur Gonca

In [27]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- valor: double (nullable = true)
 |-- parte_debitada_nome: string (nullable = true)
 |-- parte_debitada_conta: string (nullable = true)
 |-- parte_debitada_banco: string (nullable = true)
 |-- parte_creditada_nome: string (nullable = true)
 |-- parte_creditada_conta: string (nullable = true)
 |-- parte_creditada_banco: string (nullable = true)
 |-- chave_pix_tipo: string (nullable = true)
 |-- chave_pix_valor: string (nullable = true)
 |-- data_transacao: timestamp (nullable = true)



In [28]:
# 1. Descubra a quantidade total de transações na base de dados.

df.count()

100

In [29]:
#2. Qual foi a quantia total de dinheiro transacional em cada ano?

from pyspark.sql.functions import year, col

df.withColumn(
    'year', year(col('data_transacao'))
).groupBy('year').sum('valor').show()


+----+------------------+
|year|        sum(valor)|
+----+------------------+
|2022|517980.58999999973|
|2021| 848320.6000000001|
+----+------------------+



In [30]:
# 3. Descubra a quantidade de dinheiro que entrou na Nubank em todo o período.

from pyspark.sql.functions import sum

df.filter(col('parte_creditada_banco') == "Nubank").select(sum('valor')).show()




+----------+
|sum(valor)|
+----------+
| 302224.64|
+----------+



In [33]:

# 4. Retorne a quantidade de transações agrupadas por tipo de chave.

df.groupby('chave_pix_tipo').count().orderBy('count', ascending = False).show()

+--------------+-----+
|chave_pix_tipo|count|
+--------------+-----+
|           cpf|   49|
|         email|   29|
|       celular|   22|
+--------------+-----+



In [34]:
# 5. Descubra qual é a média, mínima e máxima do valor de todas as transações.

df.select('valor').describe().show()

from pyspark.sql import functions as F

df.select(
    F.max(col('valor')),
    F.min(col('valor')),
    F.mean(col('valor')),
).show()

+-------+------------------+
|summary|             valor|
+-------+------------------+
|  count|               100|
|   mean|13663.011899999998|
| stddev|25715.376055332952|
|    min|              0.03|
|    max|          95977.62|
+-------+------------------+

+----------+----------+------------------+
|max(valor)|min(valor)|        avg(valor)|
+----------+----------+------------------+
|  95977.62|      0.03|13663.011899999998|
+----------+----------+------------------+



In [35]:

# 6. Quais foram as 5 maiores transações durante todo o período?

df.orderBy('valor', ascending = False).show(5)

+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+--------------------+-------------------+
| id|   valor| parte_debitada_nome|parte_debitada_conta|parte_debitada_banco|parte_creditada_nome|parte_creditada_conta|parte_creditada_banco|chave_pix_tipo|     chave_pix_valor|     data_transacao|
+---+--------+--------------------+--------------------+--------------------+--------------------+---------------------+---------------------+--------------+--------------------+-------------------+
| 54|95977.62|   Henrique Ferreira|             6269315|              Nubank|Sra. Maria Luiza ...|             68542779|                  BTG|           cpf|         14957860211|2021-09-22 10:17:00|
| 86|94736.79|Joao Vitor Cavalc...|            52265189|                 BTG|        Stella Gomes|             93858790|               Nubank|         email|stella.gomes@yaho...|2021-01-26 00:04:00|
| 97|

In [36]:

# 7. Para o banco BTG, mostre qual é a chave pix mais utilizada para enviar ou receber transações.
df.filter((col('parte_debitada_banco') == "BTG") |(col('parte_creditada_banco') == "BTG")).groupBy('chave_pix_tipo').count().orderBy('count', ascending=False).show()

+--------------+-----+
|chave_pix_tipo|count|
+--------------+-----+
|           cpf|   22|
|         email|   15|
|       celular|   13|
+--------------+-----+



In [52]:

#8. Descubra qual foi a maior transação de cada mês na base de dados.

from pyspark.sql.functions import date_format
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number



window_func  = Window.partitionBy("ano_mes").orderBy(col('valor').desc())


df.select(
    'id', 'valor', 'data_transacao'
).withColumn(
    'ano_mes', date_format(col('data_transacao'), 'y-MM')
).withColumn('row_number', row_number().over(window_func)).filter(col('row_number') == 1).show()

+---+--------+-------------------+-------+----------+
| id|   valor|     data_transacao|ano_mes|row_number|
+---+--------+-------------------+-------+----------+
| 86|94736.79|2021-01-26 00:04:00|2021-01|         1|
| 78|81977.98|2021-02-16 18:57:00|2021-02|         1|
| 88|78347.58|2021-03-18 14:21:00|2021-03|         1|
| 44|   459.2|2021-04-06 03:47:00|2021-04|         1|
| 68|  611.53|2021-05-21 06:56:00|2021-05|         1|
| 52|60139.23|2021-06-13 16:31:00|2021-06|         1|
| 76| 9320.87|2021-07-16 00:01:00|2021-07|         1|
| 73|  570.52|2021-08-02 18:40:00|2021-08|         1|
| 54|95977.62|2021-09-22 10:17:00|2021-09|         1|
| 32| 1508.83|2021-10-12 10:04:00|2021-10|         1|
| 71|80083.34|2021-11-16 10:58:00|2021-11|         1|
| 40|  829.87|2021-12-09 06:10:00|2021-12|         1|
| 79| 78559.4|2022-01-09 22:31:00|2022-01|         1|
| 35| 7944.02|2022-02-02 14:41:00|2022-02|         1|
| 55|35409.61|2022-03-02 03:04:00|2022-03|         1|
| 31|20875.64|2022-04-29 10:

In [57]:

#9. Crie uma nova coluna que contenha o valor da transação em dólar.
from pyspark.sql.functions import round

df.withColumn('valor_dolar', round(col('valor') * 5.09, 2)).select('id', 'valor', 'valor_dolar').show()


+---+--------+-----------+
| id|   valor|valor_dolar|
+---+--------+-----------+
|  1|    9.93|      50.54|
|  2|   15.38|      78.28|
|  3|   57.58|     293.08|
|  4|53705.13|  273359.11|
|  5|25299.69|  128775.42|
|  6| 7165.06|   36470.16|
|  7|    6.16|      31.35|
|  8|  136.36|     694.07|
|  9|  574.39|    2923.65|
| 10|   42.88|     218.26|
| 11|33629.97|  171176.55|
| 12| 4374.56|   22266.51|
| 13|  507.18|    2581.55|
| 14|67758.87|  344892.65|
| 15|  815.53|    4151.05|
| 16|    2.73|       13.9|
| 17|    0.54|       2.75|
| 18|49836.72|   253668.9|
| 19|    9.68|      49.27|
| 20| 9837.22|   50071.45|
+---+--------+-----------+
only showing top 20 rows

